In [1]:
import pandas as pd
import os

base = os.getcwd()

filepath =  os.path.join(base, "Texas_AgCensus_2012_2017_2022.csv")

df = pd.DataFrame(pd.read_csv(filepath))

C:\Users\scox1\AppData\Local\Temp\ipykernel_3484\1425984748.py:8: DtypeWarning: Columns (18,21,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.DataFrame(pd.read_csv(filepath))


In [2]:
print(df['COMMODITY_DESC'].value_counts())


COMMODITY_DESC
AG LAND                         145294
CATTLE                          101384
PRODUCERS                       101333
FARM OPERATIONS                  73104
COMMODITY TOTALS                 47322
                                 ...  
FERTILIZER & CHEMICAL TOTALS         6
POPCORN                              6
MUSTARD                              6
MISCANTHUS                           6
MOLLUSKS                             4
Name: count, Length: 247, dtype: int64


In [3]:
# Identify production rows
yield_df = df[df['SHORT_DESC'].str.contains("PRODUCTION", case=False, na=False)][['COMMODITY_DESC','SHORT_DESC']]
print(yield_df['COMMODITY_DESC'].shape)



(32335,)


In [4]:
# 1. Filter for county-level rows
df = df[df['AGG_LEVEL_DESC'] == 'COUNTY'].copy()

# 2. Create unified feature name

df['FEATURE'] = (
    df['COMMODITY_DESC'].astype(str).str.strip() + "_" +
    df['SHORT_DESC'].astype(str).str.strip()
)

df['FEATURE'] = df['FEATURE'].str.replace(r'[^A-Za-z0-9_]+', '', regex=True)


df['VALUE'] = (
    df['VALUE']
    .astype(str)
    .str.replace(',', '', regex=False)   # remove commas
    .str.replace(r'[^0-9.\-]', '', regex=True)  # remove units, parentheses, etc.
)

df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')


# 4. Pivot with aggregation
wide = df.pivot_table(
    index=['COUNTY_NAME', 'YEAR'],
    columns='FEATURE',
    values='VALUE'
).reset_index()

In [5]:
wide

FEATURE,COUNTY_NAME,YEAR,AGLAND_AGLANDACRES,AGLAND_AGLANDCROPINSURANCEACRES,AGLAND_AGLANDCROPINSURANCENUMBEROFOPERATIONS,AGLAND_AGLANDCROPLANDACRES,AGLAND_AGLANDCROPLANDAREAMEASUREDINPCTOFAGLAND,AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDACRES,AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDALLCROPSFAILEDACRES,AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDALLCROPSFAILEDNUMBEROFOPERATIONS,...,WHEAT_WHEATSPRINGEXCLDURUMOPERATIONSWITHAREAHARVESTED,WHEAT_WHEATSPRINGEXCLDURUMPRODUCTIONMEASUREDINBU,WHEAT_WHEATWINTERACRESHARVESTED,WHEAT_WHEATWINTERIRRIGATEDACRESHARVESTED,WHEAT_WHEATWINTERIRRIGATEDOPERATIONSWITHAREAHARVESTED,WHEAT_WHEATWINTEROPERATIONSWITHAREAHARVESTED,WHEAT_WHEATWINTERPRODUCTIONMEASUREDINBU,WOOL_WOOLOPERATIONSWITHPRODUCTION,WOOL_WOOLPRODUCTIONMEASUREDINLB,WOOL_WOOLSALESMEASUREDIN
0,ANDERSON,2012,7391.0,10512.0,32.0,70333.0,NaN,4862.0,3134.0,85.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,ANDERSON,2017,32795.0,14907.0,14.0,63774.0,13.75,4239.5,1979.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ANDERSON,2022,32469.0,16196.0,41.0,69021.0,NaN,5632.0,2740.0,103.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,ANDREWS,2012,24662.0,23915.0,19.0,71517.0,NaN,29342.5,NaN,26.0,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,1.0,NaN,NaN
4,ANDREWS,2017,32830.0,322066.0,12.0,78257.0,8.00,35201.5,9247.0,12.0,...,NaN,NaN,1500.0,320.0,4.0,7.0,31400.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,ZAPATA,2017,15203.0,21736.0,21.0,18856.0,6.50,12208.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,ZAPATA,2022,4025.0,23413.0,17.0,5980.0,NaN,1897.5,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
759,ZAVALA,2012,295056.0,50198.0,39.0,95980.0,NaN,11980.5,4298.0,27.0,...,1.0,NaN,NaN,NaN,10.0,23.0,NaN,1.0,NaN,NaN
760,ZAVALA,2017,401018.0,57149.0,39.0,76046.0,9.50,15045.0,1731.0,12.0,...,2.0,NaN,NaN,NaN,13.0,19.0,NaN,NaN,NaN,NaN


In [6]:
print(wide.columns)

Index(['COUNTY_NAME', 'YEAR', 'AGLAND_AGLANDACRES',
       'AGLAND_AGLANDCROPINSURANCEACRES',
       'AGLAND_AGLANDCROPINSURANCENUMBEROFOPERATIONS',
       'AGLAND_AGLANDCROPLANDACRES',
       'AGLAND_AGLANDCROPLANDAREAMEASUREDINPCTOFAGLAND',
       'AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDACRES',
       'AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDALLCROPSFAILEDACRES',
       'AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDALLCROPSFAILEDNUMBEROFOPERATIONS',
       ...
       'WHEAT_WHEATSPRINGEXCLDURUMOPERATIONSWITHAREAHARVESTED',
       'WHEAT_WHEATSPRINGEXCLDURUMPRODUCTIONMEASUREDINBU',
       'WHEAT_WHEATWINTERACRESHARVESTED',
       'WHEAT_WHEATWINTERIRRIGATEDACRESHARVESTED',
       'WHEAT_WHEATWINTERIRRIGATEDOPERATIONSWITHAREAHARVESTED',
       'WHEAT_WHEATWINTEROPERATIONSWITHAREAHARVESTED',
       'WHEAT_WHEATWINTERPRODUCTIONMEASUREDINBU',
       'WOOL_WOOLOPERATIONSWITHPRODUCTION', 'WOOL_WOOLPRODUCTIONMEASUREDINLB',
       'WOOL_WOOLSALESMEASUREDIN'],
      dtype='object', name='FEA

In [7]:
nan_pct = wide.isna().mean().sort_values(ascending=False)
print(nan_pct.head(20))  # Top 20 most sparse features


FEATURE
MILLET_MILLETPROSOIRRIGATEDACRESHARVESTED                                         0.998688
RASPBERRIES_RASPBERRIESBLACKACRESNONBEARING                                       0.998688
PRUNES_PRUNESOPERATIONSWITHAREANONBEARING                                         0.998688
PRUNES_PRUNESOPERATIONSWITHAREABEARING                                            0.998688
ARONIABERRIES_ARONIABERRIESOPERATIONSWITHAREANONBEARING                           0.998688
CURRANTS_CURRANTSACRESGROWN                                                       0.998688
CURRANTS_CURRANTSOPERATIONSWITHAREABEARING                                        0.998688
PROPAGATIVEMATERIAL_PROPAGATIVEMATERIALINTHEOPENOPERATIONSWITHAREAINPRODUCTION    0.998688
DATES_DATESACRESBEARING                                                           0.998688
DATES_DATESACRESBEARINGNONBEARING                                                 0.998688
ELDERBERRIES_ELDERBERRIESACRESNONBEARING                                          

In [8]:
for c in wide.columns:
    if('PRODUCTION' in c.upper()):
        print(c)

ANIMALSOTHER_ANIMALSOTHERPRODUCTIONCONTRACTOPERATIONSWITHPRODUCTION
AQUATICPLANTS_AQUATICPLANTSINTHEOPENACRESINPRODUCTION
AQUATICPLANTS_AQUATICPLANTSOPERATIONSWITHAREAINPRODUCTION
AQUATICPLANTS_AQUATICPLANTSUNDERPROTECTIONSQFTINPRODUCTION
BARLEY_BARLEYPRODUCTIONMEASUREDINBU
BEANS_BEANSDRYEDIBLEEXCLCHICKPEASLIMAPRODUCTIONMEASUREDINCWT
BEANS_BEANSDRYEDIBLEEXCLLIMAPRODUCTIONMEASUREDINCWT
BEDDINGPLANTTOTALS_BEDDINGPLANTTOTALSINTHEOPENACRESINPRODUCTION
BEDDINGPLANTTOTALS_BEDDINGPLANTTOTALSINTHEOPENOPERATIONSWITHAREAINPRODUCTION
BEDDINGPLANTTOTALS_BEDDINGPLANTTOTALSOPERATIONSWITHAREAINPRODUCTION
BEDDINGPLANTTOTALS_BEDDINGPLANTTOTALSUNDERPROTECTIONSQFTINPRODUCTION
BULBSCORMSRHIZOMESTUBERS_BULBSCORMSRHIZOMESTUBERSDRYINTHEOPENACRESINPRODUCTION
BULBSCORMSRHIZOMESTUBERS_BULBSCORMSRHIZOMESTUBERSDRYOPERATIONSWITHAREAINPRODUCTION
BULBSCORMSRHIZOMESTUBERS_BULBSCORMSRHIZOMESTUBERSDRYUNDERPROTECTIONSQFTINPRODUCTION
CANOLA_CANOLAPRODUCTIONMEASUREDINLB
CATTLE_CATTLEHEIFERSGE500LBSMILKREPLACEMENTPRODUCTIO

In [9]:
"""#remove columns with more than 80% NaN data
threshold = 0.8
wide = wide.loc[:, wide.isna().mean() < threshold]
wide"""

'#remove columns with more than 80% NaN data\nthreshold = 0.8\nwide = wide.loc[:, wide.isna().mean() < threshold]\nwide'

In [10]:
# Fill per county using rolling median or median over years
wide_transformed = wide.groupby("COUNTY_NAME").transform(lambda x: x.fillna(x.median()))

# put COUNTY_NAME back
wide_transformed["COUNTY_NAME"] = wide["COUNTY_NAME"]
wide = wide_transformed


c:\Users\scox1\anaconda3\envs\tf\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\scox1\anaconda3\envs\tf\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\scox1\anaconda3\envs\tf\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\scox1\anaconda3\envs\tf\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\scox1\anaconda3\envs\tf\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\scox1\anaconda3\envs\tf\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
 

In [12]:
wide_transformed.head()

FEATURE,YEAR,AGLAND_AGLANDACRES,AGLAND_AGLANDCROPINSURANCEACRES,AGLAND_AGLANDCROPINSURANCENUMBEROFOPERATIONS,AGLAND_AGLANDCROPLANDACRES,AGLAND_AGLANDCROPLANDAREAMEASUREDINPCTOFAGLAND,AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDACRES,AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDALLCROPSFAILEDACRES,AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDALLCROPSFAILEDNUMBEROFOPERATIONS,AGLAND_AGLANDCROPLANDEXCLHARVESTEDPASTUREDCULTIVATEDSUMMERFALLOWACRES,...,WHEAT_WHEATSPRINGEXCLDURUMPRODUCTIONMEASUREDINBU,WHEAT_WHEATWINTERACRESHARVESTED,WHEAT_WHEATWINTERIRRIGATEDACRESHARVESTED,WHEAT_WHEATWINTERIRRIGATEDOPERATIONSWITHAREAHARVESTED,WHEAT_WHEATWINTEROPERATIONSWITHAREAHARVESTED,WHEAT_WHEATWINTERPRODUCTIONMEASUREDINBU,WOOL_WOOLOPERATIONSWITHPRODUCTION,WOOL_WOOLPRODUCTIONMEASUREDINLB,WOOL_WOOLSALESMEASUREDIN,COUNTY_NAME
0,2012,7391.0,10512.0,32.0,70333.0,13.75,4862.0,3134.0,85.0,858.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,ANDERSON
1,2017,32795.0,14907.0,14.0,63774.0,13.75,4239.5,1979.0,59.0,2112.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,ANDERSON
2,2022,32469.0,16196.0,41.0,69021.0,13.75,5632.0,2740.0,103.0,3542.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,ANDERSON
3,2012,24662.0,23915.0,19.0,71517.0,8.00,29342.5,9247.0,26.0,7806.0,...,NaN,1500.0,320.0,2.0,2.0,31400.0,1.0,2250.0,NaN,ANDREWS
4,2017,32830.0,322066.0,12.0,78257.0,8.00,35201.5,9247.0,12.0,7806.0,...,NaN,1500.0,320.0,4.0,7.0,31400.0,2.0,2250.0,NaN,ANDREWS


In [13]:
wide_transformed.to_csv("cleaned_COUNTY_data.csv")